# 1/ Préparation des données (identique au TP, sans les graphiques)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn import preprocessing
from sklearn import neighbors, metrics

data = pd.read_csv('winequality-white.csv', sep=";")

X = data.as_matrix(data.columns[:-1])
y = data.as_matrix([data.columns[-1]])
y = y.flatten()

y_class = np.where(y<6, 0, 1)

X_train, X_test, y_train, y_test = \
    model_selection.train_test_split(X, y_class, 
                                     test_size=0.3 # 30% des données dans le jeu de test
                                    )

std_scale = preprocessing.StandardScaler().fit(X_train)
X_train_std = std_scale.transform(X_train)
X_test_std = std_scale.transform(X_test)

# 2/ Résultats du TP avec la classe GridSearchCV de Scikit

In [8]:
# Code du TP avec la classe GridSearchCV de Scikit

# Fixer les valeurs des hyperparamètres à tester
param_grid = {'n_neighbors':[3, 5, 7, 9, 11, 13, 15]}

# Choisir un score à optimiser, ici l'accuracy (proportion de prédictions correctes)
score = 'accuracy'

# Créer un classifieur kNN avec recherche d'hyperparamètre par validation croisée
clf = model_selection.GridSearchCV(neighbors.KNeighborsClassifier(), # un classifieur kNN
param_grid, # hyperparamètres à tester
cv=5, # nombre de folds de validation croisée
scoring=score # score à optimiser
)

# Optimiser ce classifieur sur le jeu d'entraînement
# clf.fit(X_train, y_train)
clf.fit(X_train_std, y_train)

# Afficher le(s) hyperparamètre(s) optimaux
print("Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:")
print(clf.best_params_)

# Afficher les performances correspondantes
print("Résultats de la validation croisée :")
for mean, std, params in zip(clf.cv_results_['mean_test_score'], # score moyen
clf.cv_results_['std_test_score'], # écart-type du score
clf.cv_results_['params'] # valeur de l'hyperparamètre
):
    print("\t%s = %0.3f (+/-%0.03f) for %r" % (
        score, # critère utilisé 
        mean, # score moyen 
        std * 2, # barre d'erreur 
        params # hyperparamètre 
        ))

y_pred_scikit = clf.predict(X_test_std)
print("\nSur le jeu de test : %0.3f" % metrics.accuracy_score(y_test, y_pred_scikit))

Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:
{'n_neighbors': 15}
Résultats de la validation croisée :
	accuracy = 0.749 (+/-0.034) for {'n_neighbors': 3}
	accuracy = 0.750 (+/-0.024) for {'n_neighbors': 5}
	accuracy = 0.748 (+/-0.025) for {'n_neighbors': 7}
	accuracy = 0.754 (+/-0.031) for {'n_neighbors': 9}
	accuracy = 0.756 (+/-0.028) for {'n_neighbors': 11}
	accuracy = 0.754 (+/-0.030) for {'n_neighbors': 13}
	accuracy = 0.758 (+/-0.022) for {'n_neighbors': 15}

Sur le jeu de test : 0.776


# 3/ Nouveaux résultats avec la classe créée : KnnGridSearchCV

#### Cette classe s'utilise de la même manière que celle de Scikit, à savoir:
_ Initialisation avec la liste des paramètres à tester et le nombre de folds

_ Une fonction fit pour tester les différents paramètres et trouver le meilleur

_ Une fonction predict pour appliquer le meilleur modèle trouvé avec fit à un jeu de test

In [9]:
from KnnGridSearchCV import KnnGridSearchCV

# Fixer les valeurs des hyperparamètres à tester
param_grid = {'n_neighbors':[3, 5, 7, 9, 11, 13, 15]}

# Ici la classe créée ne prend en paramètres que la liste des hyperparamètres
# et le nombre de folds
clf = KnnGridSearchCV(param_grid['n_neighbors'], # hyperparamètres à tester
                      nb_folds=5, # nombre de folds de validation croisée
                     )

# Optimiser ce classifieur sur le jeu d'entraînement
clf.fit(X_train_std, y_train)

# Afficher le(s) hyperparamètre(s) optimaux
print("Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:")
print(clf.best_params_)

# Afficher les performances correspondantes
print("Résultats de la validation croisée :")
for mean, std, params in zip(clf.cv_results_['mean_test_score'], # score moyen
clf.cv_results_['std_test_score'], # écart-type du score
clf.cv_results_['params'] # valeur de l'hyperparamètre
):
    print("\t%s = %0.3f (+/-%0.03f) for %r" % (
        score, # critère utilisé 
        mean, # score moyen 
        std * 2, # barre d'erreur 
        params # hyperparamètre 
        ))

y_pred_new = clf.predict(X_test_std)
print("\nSur le jeu de test : %0.3f" % metrics.accuracy_score(y_test, y_pred_new))

Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:
15
Résultats de la validation croisée :
	accuracy = 0.749 (+/-0.034) for 3
	accuracy = 0.750 (+/-0.024) for 5
	accuracy = 0.748 (+/-0.025) for 7
	accuracy = 0.754 (+/-0.031) for 9
	accuracy = 0.756 (+/-0.028) for 11
	accuracy = 0.754 (+/-0.030) for 13
	accuracy = 0.758 (+/-0.022) for 15

Sur le jeu de test : 0.776


### Les résultats sont identiques mais on peut vérifier aussi que les deux fonctions donnent bien le même résultat final sur le jeu de test

In [10]:
print(metrics.accuracy_score(y_test, y_pred_scikit) == metrics.accuracy_score(y_test, y_pred_new))

True
